In [1]:
from bs4 import BeautifulSoup
import time

import pandas as pd
from pandas import DataFrame, Series

import os

In [2]:
##############################################
#id = {'0505562017780', '0515559000286', '0903555001614'}


In [3]:
#info fn
#input = info.html, ENG.txt // output = [no.rows, dict]
def info(infoHtml, engTxt):
    
    ########### setup ########
    with open(infoHtml, encoding='utf8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    #output
    out=dict()
    #define id_ (string)
    id_=soup.find(class_='col-md-4').p.string.split(':')[1].strip()
    #find all years (int)
    years=sorted([int(i.text) for i in soup.find_all('td')[9].find_all('a')])
    miny=years[0]
    years.append(miny-1)
    years.append(miny-2)
    years=sorted(years)
    
    ######## primary keys ########
    #id_
    out['เลขทะเบียนนิติบุคคล']=[id_]*len(years)
    #years
    out['ปีที่ส่งงบการเงิน']=years
    
    ####################### columns ################
    #company name th
    out['ชื่อนิติบุคคล']=[soup.find("h2", class_="purple").string.split(':')[1].strip()]*len(years)
    #company name eng
    with open(engTxt, 'r') as f:
        lines=f.readlines()
    for items in [(i.strip()[:14],i.strip()[14:]) for i in lines]:
        if items[0].strip() == id_+',':
            out['ชื่อนิติบุคคล ENG']=[items[1].strip()]*len(years)
            break
    f.close()
    #company type
    out['ประเภทนิติบุคคล']=[soup.find(class_='box-title light hidden-xs').text.strip()]*len(years)
    
    #status, setup date, setup fund, old id, location, phone, phone2, web, email
    for i in range(7):
        if i%2==0:
            out[soup.find_all('td')[i].text.strip()]=[soup.find_all('td')[i+1].text.strip()]*len(years)
    #location
    out['ที่ตั้ง']= [soup.find_all('td')[12].text.strip()]*len(years)
    #others
    for i in range(13,20,1):
        if i%2==1:
            out[soup.find_all('td')[i].text.strip()]=[soup.find_all('td')[i+1].text.strip()]*len(years)
            
    #other data in the table
    out['รายชื่อคณะกรรมการ']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[0].find_all('li') if len(i.text.strip())!=0])]*len(years)
    out['กรรมการลงชื่อผูกพัน']=[soup.find_all(class_='box-text')[1].text.replace('\n','').replace('\t','')]*len(years)
    out['เลขประเภทธุรกิจตอนจดทะเบียน']=[soup.find_all(class_='box-text')[2].p.text.strip().split(' ')[0].strip()]*len(years)
    out['คำอธิบายประเภทธุรกิจตอนจดทะเบียน']=[soup.find_all(class_='box-text')[2].p.text.strip().split(' ')[1].strip()]*len(years)
    out['วัตถุประสงค์ตอนจดทะเบียน']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[3].find_all('p') if len(i.text.strip())!=0])]*len(years)
    out['เลขประเภทธุรกิจที่ส่งงบการเงินปีล่าสุด']=[soup.find_all(class_='box-text')[4].p.text.strip().split(' ')[0].strip()]*len(years)
    out['คำอธิบายประเภทธุรกิจที่ส่งงบการเงินปีล่าสุด']=[soup.find_all(class_='box-text')[4].p.text.strip().split(' ')[1].strip()]*len(years)
    out['วัตถุประสงค์ที่ส่งงบการเงินปีล่าสุด']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[5].find_all('p') if len(i.text.strip())!=0])]*len(years)
    out['เครื่องหมายการค้า']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[8].find_all('span') if len(i.text.strip())!=0])]*len(years)
    out['ข้อมูลการเปลี่ยนแปลงสถานะ']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[11].find_all('p') if len(i.text.strip()) != 0])]*len(years)
    out['ข้อควรทราบ']=[soup.find_all(class_='box-text')[12].text.strip()]*len(years)
    out['สถานะอื่นๆ']=[soup.find_all(class_='box-text')[9].text.strip()]*len(years)
    
    fp.close()
    
    return [years, out]

#info only_fn
#input = info.html, invest,emt.html // output = dict
def only_info(infoHtml, engTxt, investmentHtml):
    
    ########### setup ########
    with open(infoHtml, encoding='utf8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    #output
    out=dict()
    #define id_ (string)
    id_=soup.find(class_='col-md-4').p.string.split(':')[1].strip()
    
    ######## primary keys ########
    #id_
    out['เลขทะเบียนนิติบุคคล']=[id_]

    
    ####################### columns ################
    #company name th
    out['ชื่อนิติบุคคล']=[soup.find("h2", class_="purple").string.split(':')[1].strip()]
    #company name eng
    with open(engTxt, 'r') as f:
        lines=f.readlines()
    for items in [(i.strip()[:14],i.strip()[14:]) for i in lines]:
        if items[0].strip() == id_+',':
            out['ชื่อนิติบุคคล ENG']=[items[1].strip()]
            break
    f.close()
    #company type
    out['ประเภทนิติบุคคล']=[soup.find(class_='box-title light hidden-xs').text.strip()]
    
    #status, setup date, setup fund, old id, location, phone, phone2, web, email
    for i in range(7):
        if i%2==0:
            out[soup.find_all('td')[i].text.strip()]=[soup.find_all('td')[i+1].text.strip()]
    #location
    out['ที่ตั้ง']= [soup.find_all('td')[12].text.strip()]
    #others
    for i in range(13,20,1):
        if i%2==1:
            out[soup.find_all('td')[i].text.strip()]=[soup.find_all('td')[i+1].text.strip()]
            
    #other data in the table
    out['รายชื่อคณะกรรมการ']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[0].find_all('li') if len(i.text.strip())!=0])]
    out['กรรมการลงชื่อผูกพัน']=[soup.find_all(class_='box-text')[1].text.replace('\n','').replace('\t','')]
    out['เลขประเภทธุรกิจตอนจดทะเบียน']=[soup.find_all(class_='box-text')[2].p.text.strip().split(' ')[0].strip()]
    out['คำอธิบายประเภทธุรกิจตอนจดทะเบียน']=[soup.find_all(class_='box-text')[2].p.text.strip().split(' ')[1].strip()]
    out['วัตถุประสงค์ตอนจดทะเบียน']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[3].find_all('p') if len(i.text.strip())!=0])]
    out['เลขประเภทธุรกิจที่ส่งงบการเงินปีล่าสุด']=[soup.find_all(class_='box-text')[4].p.text.strip().split(' ')[0].strip()]
    out['คำอธิบายประเภทธุรกิจที่ส่งงบการเงินปีล่าสุด']=[soup.find_all(class_='box-text')[4].p.text.strip().split(' ')[1].strip()]
    out['วัตถุประสงค์ที่ส่งงบการเงินปีล่าสุด']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[5].find_all('p') if len(i.text.strip())!=0])]
    out['เครื่องหมายการค้า']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[8].find_all('span') if len(i.text.strip())!=0])]
    out['ข้อมูลการเปลี่ยนแปลงสถานะ']=[', '.join([i.text.strip().replace('\n','').replace('\t','') for i in soup.find_all(class_='box-text')[11].find_all('p') if len(i.text.strip()) != 0])]
    out['ข้อควรทราบ']=[soup.find_all(class_='box-text')[12].text.strip()]
    out['สถานะอื่นๆ']=[soup.find_all(class_='box-text')[9].text.strip()]
    
    fp.close()
    
    inv_dict=investment(investmentHtml)
    for key in inv_dict:
        out[key]=inv_dict[key]
    
    return out

#input = balance.html // output = [years, dict]
def balance(balanceHtml):
    with open(balanceHtml, encoding='utf8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    years=sorted([int(i.text.strip()) for i in soup.find_all('th')[1:4]])
    temp=dict()
    kk=soup.find_all('td')[:77]
    for i in range(77):
        if i%7 == 0:
            temp[kk[i].text.strip()]=[kk[i+1].text.strip(), kk[i+3].text.strip(), kk[i+5].text.strip()]
            temp[kk[i].text.strip()+' %เปลี่ยนแปลง']=[kk[i+2].text.strip(), kk[i+4].text.strip(), kk[i+6].text.strip()]
    fp.close()
    return [years, temp]

#input = profit.html // output = [years, dict]
def profit(profitHtml):
    with open(profitHtml, encoding='utf8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    years=sorted([int(i.text.strip()) for i in soup.find_all('th')[1:4]])
    temp=dict()
    kk=soup.find_all('td')[:70]
    for i in range(70):
        if i%7 == 0:
            temp[kk[i].text.strip()]=[kk[i+1].text.strip(), kk[i+3].text.strip(), kk[i+5].text.strip()]
            temp[kk[i].text.strip()+' %เปลี่ยนแปลง']=[kk[i+2].text.strip(), kk[i+4].text.strip(), kk[i+6].text.strip()]
    fp.close()
    return [years, temp]

#input = ratio.html // output = [years, dict]
def ratio(ratioHtml):
    with open(ratioHtml, encoding='utf8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    years=sorted([int(i.text.strip()) for i in soup.find_all('th')[2:5]])
    temp=dict()
    kk=soup.find_all('td')[:75]
    for i in range(75):
        if i%5 == 0:
            temp[kk[i+1].text.strip()]=[kk[i+2].text.strip(), kk[i+3].text.strip(), kk[i+4].text.strip()]
    fp.close()
    return [years, temp]

#input = investment.html // output = dict
def investment(investmentHtml):
    with open(investmentHtml, encoding='utf8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    temp=dict()
    kk=soup.find_all('td')
    for i in range(len(kk)):
        if kk[i].text.strip()=='ไทย':
            stock_th=kk[i+1].text.strip().replace(',','')
            value_th=kk[i+2].text.strip().replace(',','')
            ratio_th=kk[i+3].text.strip().replace(',','')
            temp['จำนวนหุ้นในไทย (หุ้น)']=[stock_th]
            temp['มูลค่าการลงทุนในไทย (บาท)']=[value_th]
            temp['สัดส่วนมูลค่าการลงทุนในไทย (%)']=[ratio_th]
        if kk[i].text.strip()=='รวมทุกสัญชาติ':
            stock_oth=float(kk[i+1].text.strip().replace(',',''))-float(stock_th)
            value_oth=float(kk[i+2].text.strip().replace(',',''))-float(value_th)
            ratio_oth=float(100)-float(ratio_th)
            temp['จำนวนหุ้นในต่างชาติ (หุ้น)']=[str(stock_oth)]
            temp['มูลค่าการลงทุนในต่างชาติ (บาท)']=[str(value_oth)]
            temp['สัดส่วนมูลค่าการลงทุนในต่างชาติ (%)']=[str(ratio_oth)]
    fp.close()
    return temp



In [5]:
#test only_info()
infoHtml='C:\\Users\\saeth\\Desktop\\ext3\\46313\\0505562017780_info.html'
engTxt='C:\\Users\\saeth\\Downloads\\46313_eng.txt'
investmentHtml='C:\\Users\\saeth\\Desktop\\ext3\\46313\\0505562017780_invest.html'
only_info(infoHtml, engTxt, investmentHtml)

{'เลขทะเบียนนิติบุคคล': ['0505562017780'],
 'ชื่อนิติบุคคล': ['บริษัท 123 พรีเมี่ยม ฟรุ๊ต จำกัด'],
 'ชื่อนิติบุคคล ENG': ['123 PREMIUM FRUIT CO., LTD.'],
 'ประเภทนิติบุคคล': ['บริษัทจำกัด'],
 'สถานะนิติบุคคล': ['ยังดำเนินกิจการอยู่'],
 'วันที่จดทะเบียนจัดตั้ง': ['05 พ.ย. 2562'],
 'ทุนจดทะเบียน': ['5,000,000.00 บาท'],
 'เลขทะเบียนเดิม': ['-'],
 'ที่ตั้ง': ['271  หมู่ที่ 7 ต.บ้านดู่ อ.เมืองเชียงราย จ.เชียงราย'],
 'โทรศัพท์': ['-'],
 'โทรสาร': ['-'],
 'Website': ['-'],
 'E-mail address': ['M_anong@hotmail.co.th'],
 'รายชื่อคณะกรรมการ': ['นางธัญณีย์ ธรรมเบญจโรจน์, นายวสุวัตน์ เลาหวิวัฒน์กุล/'],
 'กรรมการลงชื่อผูกพัน': ['กรรมการหนึ่งคนลงลายมือชื่อ และประทับตราสำคัญของบริษัท/'],
 'เลขประเภทธุรกิจตอนจดทะเบียน': ['46313'],
 'คำอธิบายประเภทธุรกิจตอนจดทะเบียน': ['การขายส่งผักและผลไม้'],
 'วัตถุประสงค์ตอนจดทะเบียน': ['ประกอบกิจการส่งออกสับปะรด'],
 'เลขประเภทธุรกิจที่ส่งงบการเงินปีล่าสุด': ['46313'],
 'คำอธิบายประเภทธุรกิจที่ส่งงบการเงินปีล่าสุด': ['การขายส่งผักและผลไม้'],
 'วัตถุประสงค์ที่ส่งงบกา

In [10]:
path='C:\\Users\\saeth\\Desktop\\ext3\\46313\\'
path_eng='C:\\Users\\saeth\\Downloads\\46313_eng.txt'
files=sorted(os.listdir(path))
unique_id=set([i[:13] for i in files])
frame=[]
only_info_frame=[]
for id_ in unique_id:
    out=dict()
    temp=[i for i in files if i[:13]==id_]
    temp_id= id_+'_info.html'
    temp_invest= id_+'_invest.html'
    temp_b=sorted([i for i in temp if '_b' in i[13:]])
    temp_p=sorted([i for i in temp if '_p' in i[13:]])
    temp_r=sorted([i for i in temp if '_r' in i[13:]])
    
    #only_info
    for key in only_info(path+temp_id, path_eng, path+temp_invest).keys():
        out[key]=only_info(path+temp_id, path_eng, path+temp_invest)[key]
    only_info_frame.append(pd.DataFrame(out))
    
    #has years
    if len(temp_b)!=0:
        out=dict()
        #info()
        #print(info(path+temp_id, path_eng))
        info_dict=info(path+temp_id, path_eng)[1]
        years=info(path+temp_id, path_eng)[0]
        for key in info_dict.keys():
            out[key]=info_dict[key]

        #balance
        temp_balance=dict()
        for b in temp_b:
            b_dict=balance(path+b)[1]
            for key in b_dict.keys():
                if key not in temp_balance.keys():
                    temp_balance[key]=b_dict[key]
                else:
                    if len(years)-len(temp_balance[key])>=3:
                        temp_balance[key]=temp_balance[key]+b_dict[key]
                    else:
                        dif=len(years)-len(temp_balance[key])
                        if dif==1:
                            tar=[b_dict[key][-1]]
                        else:
                            tar=b_dict[key][1:]
                        temp_balance[key]=temp_balance[key]+tar
        for key in temp_balance.keys():
            out[key]=temp_balance[key]

        #profit
        temp_profit=dict()
        for b in temp_p:
            b_dict=profit(path+b)[1]
            for key in b_dict.keys():
                if key not in temp_profit.keys():
                    temp_profit[key]=b_dict[key]
                else:
                    if len(years)-len(temp_profit[key])>=3:
                        temp_profit[key]=temp_profit[key]+b_dict[key]
                    else:
                        dif=len(years)-len(temp_profit[key])
                        if dif==1:
                            tar=[b_dict[key][-1]]
                        else:
                            tar=b_dict[key][1:]
                        temp_profit[key]=temp_profit[key]+tar
        for key in temp_profit.keys():
            out[key]=temp_profit[key]

        #ratio
        temp_ratio=dict()
        for b in temp_r:
            b_dict=ratio(path+b)[1]
            for key in b_dict.keys():
                if key not in temp_ratio.keys():
                    temp_ratio[key]=b_dict[key]
                else:
                    if len(years)-len(temp_ratio[key])>=3:
                        temp_ratio[key]=temp_ratio[key]+b_dict[key]
                    else:
                        dif=len(years)-len(temp_ratio[key])
                        if dif==1:
                            tar=[b_dict[key][-1]]
                        else:
                            tar=b_dict[key][1:]
                        temp_ratio[key]=temp_ratio[key]+tar
        for key in temp_ratio.keys():
            out[key]=temp_ratio[key]

        #investment
        inv_dict=investment(path+temp_invest)
        for key in inv_dict.keys():
            out[key]=inv_dict[key]*len(years)


        #frame.append([len(i) for i in out.values()])
        #print(out)
        frame.append(pd.DataFrame(out))
    
if len(frame)!=0:
    result=pd.concat(frame,ignore_index=True)
if len(only_info_frame) !=0:
    only_info_result=pd.concat(only_info_frame,ignore_index=True)

In [11]:
result.head()

,เลขทะเบียนนิติบุคคล,ปีที่ส่งงบการเงิน,ชื่อนิติบุคคล,ชื่อนิติบุคคล ENG,ประเภทนิติบุคคล,สถานะนิติบุคคล,วันที่จดทะเบียนจัดตั้ง,ทุนจดทะเบียน,เลขทะเบียนเดิม,ที่ตั้ง,...,อัตราส่วนสินทรัพย์รวมต่อส่วนของผู้ถือหุ้น (เท่า),อัตราส่วนหนี้สินรวมต่อสินทรัพย์รวม (เท่า),อัตราส่วนหนี้สินรวมต่อส่วนของผู้ถือหุ้น (เท่า),อัตราส่วนหนี้สินรวมต่อทุนดำเนินงาน (เท่า),จำนวนหุ้นในไทย (หุ้น),มูลค่าการลงทุนในไทย (บาท),สัดส่วนมูลค่าการลงทุนในไทย (%),จำนวนหุ้นในต่างชาติ (หุ้น),มูลค่าการลงทุนในต่างชาติ (บาท),สัดส่วนมูลค่าการลงทุนในต่างชาติ (%)
0,0515559000286,2557,บริษัท 558 อินเตอร์ฟรุ๊ต จำกัด,unreal V1,บริษัทจำกัด,ยังดำเนินกิจการอยู่,01 เม.ย. 2559,"1,000,000.00 บาท",-,288 หมู่ที่ 3 ต.นครเจดีย์ อ.ป่าซาง จ.ลำพูน,...,N/A,N/A,N/A,N/A,10000.00,1000000.00,100.00,0.0,0.0,0.0
1,0515559000286,2558,บริษัท 558 อินเตอร์ฟรุ๊ต จำกัด,unreal V1,บริษัทจำกัด,ยังดำเนินกิจการอยู่,01 เม.ย. 2559,"1,000,000.00 บาท",-,288 หมู่ที่ 3 ต.นครเจดีย์ อ.ป่าซาง จ.ลำพูน,...,N/A,N/A,N/A,N/A,10000.00,1000000.00,100.00,0.0,0.0,0.0
2,0515559000286,2559,บริษัท 558 อินเตอร์ฟรุ๊ต จำกัด,unreal V1,บริษัทจำกัด,ยังดำเนินกิจการอยู่,01 เม.ย. 2559,"1,000,000.00 บาท",-,288 หมู่ที่ 3 ต.นครเจดีย์ อ.ป่าซาง จ.ลำพูน,...,1.02,0.02,0.02,0.02,10000.00,1000000.00,100.00,0.0,0.0,0.0
3,0515559000286,2560,บริษัท 558 อินเตอร์ฟรุ๊ต จำกัด,unreal V1,บริษัทจำกัด,ยังดำเนินกิจการอยู่,01 เม.ย. 2559,"1,000,000.00 บาท",-,288 หมู่ที่ 3 ต.นครเจดีย์ อ.ป่าซาง จ.ลำพูน,...,1.03,0.03,0.03,0.03,10000.00,1000000.00,100.00,0.0,0.0,0.0
4,0515559000286,2561,บริษัท 558 อินเตอร์ฟรุ๊ต จำกัด,unreal V1,บริษัทจำกัด,ยังดำเนินกิจการอยู่,01 เม.ย. 2559,"1,000,000.00 บาท",-,288 หมู่ที่ 3 ต.นครเจดีย์ อ.ป่าซาง จ.ลำพูน,...,1.07,0.06,0.07,0.06,10000.00,1000000.00,100.00,0.0,0.0,0.0


In [12]:
only_info_result.head()

,เลขทะเบียนนิติบุคคล,ชื่อนิติบุคคล,ชื่อนิติบุคคล ENG,ประเภทนิติบุคคล,สถานะนิติบุคคล,วันที่จดทะเบียนจัดตั้ง,ทุนจดทะเบียน,เลขทะเบียนเดิม,ที่ตั้ง,โทรศัพท์,...,เครื่องหมายการค้า,ข้อมูลการเปลี่ยนแปลงสถานะ,ข้อควรทราบ,สถานะอื่นๆ,จำนวนหุ้นในไทย (หุ้น),มูลค่าการลงทุนในไทย (บาท),สัดส่วนมูลค่าการลงทุนในไทย (%),จำนวนหุ้นในต่างชาติ (หุ้น),มูลค่าการลงทุนในต่างชาติ (บาท),สัดส่วนมูลค่าการลงทุนในต่างชาติ (%)
0,0515559000286,บริษัท 558 อินเตอร์ฟรุ๊ต จำกัด,unreal V1,บริษัทจำกัด,ยังดำเนินกิจการอยู่,01 เม.ย. 2559,"1,000,000.00 บาท",-,288 หมู่ที่ 3 ต.นครเจดีย์ อ.ป่าซาง จ.ลำพูน,-,...,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,10000.00,1000000.00,100.00,0.0,0.0,0.0
1,0505562017780,บริษัท 123 พรีเมี่ยม ฟรุ๊ต จำกัด,"123 PREMIUM FRUIT CO., LTD.",บริษัทจำกัด,ยังดำเนินกิจการอยู่,05 พ.ย. 2562,"5,000,000.00 บาท",-,271 หมู่ที่ 7 ต.บ้านดู่ อ.เมืองเชียงราย จ.เชี...,-,...,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,50000.00,5000000.00,100.00,0.0,0.0,0.0
2,0903555001614,ห้างหุ้นส่วนจำกัด 555 ฟาร์ม อโกร,555 FARM AGRO LIMITED PARTNERSHIP,ห้างหุ้นส่วนจำกัด,ยังดำเนินกิจการอยู่,18 มิ.ย. 2555,"1,000,000.00 บาท",-,151 ซอย23(นิพัทธ์สงเคราะห์ 1) ต.หาดใหญ่ อ.หาด...,-,...,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,1000000.00,1000000.00,100.00,0.0,0.0,0.0


In [13]:
save_path='C:\\Users\\saeth\\Downloads\\'
#result.to_csv(save_path+'dbdV1.csv')
result.to_excel(save_path+'dbdV1.xlsx')
only_info_result.to_excel(save_path+'dbd_only_info_V1.xlsx')